In [ ]:
#copy a small ttgamma and ttbar file locally
!xrdcp root://cmseos.fnal.gov//store/user/lpctop/TTGamma_FullRun2/DannyTests/TTGamma_SingleLept_2016_skimTest.root .
!xrdcp root://cmseos.fnal.gov//store/user/lpctop/TTGamma_FullRun2/DannyTests/TTbarPowheg_Semilept_2016_skimTest.root .
    

In [ ]:
from coffea import hist, util
from coffea.analysis_objects import JaggedCandidateArray

import coffea.processor as processor

In [ ]:
class TTGammaCutflow(processor.ProcessorABC):
    def __init__(self):

        
        ### Muon pt
        #Declare an axis for the dataset
        dataset_axis = hist.Cat("dataset","Dataset")
        
        #Declare an axis for the muon pt
        muon_pt_axis = hist.Bin("pt","$p_{T} [GeV]", 40, 0, 200)
        

        #Define accumulator object, a dictionary storing 
        self._accumulator = processor.dict_accumulator({
            'muon_pt': hist.Hist("Counts", dataset_axis, muon_pt_axis),
        }
        )
    
        ######
        ### Step 2. Add a histogram for the muon eta
        ######

        
    @property
    def accumulator(self):
        return self._accumulator

    def process(self, df):
        output = self.accumulator.identity()

        dataset = df['dataset']
        
        muons = JaggedCandidateArray.candidatesfromcounts(
            df['nMuon'],
            pt=df['Muon_pt'],
            eta=df['Muon_eta'],
            phi=df['Muon_phi'],
            mass=df['Muon_mass'],
            charge=df['Muon_charge'],
            relIso=df['Muon_pfRelIso04_all'],
            tightId=df['Muon_tightId'],
        )
            
        #add in jagged candidate array for electrons
        #add jagged candidates for jets
        
        
        #select muons with pt >30, eta < 2.4, tight ID, and relIso < 0.15
        muonSelectTight = ((muons.pt>30) &
                           (abs(muons.eta)<2.4) &
                           (muons.tightId) &
                           (muons.relIso < 0.15)
                          )

        tightMuons = muons[muonSelectTight]
        
        #select events with exactly one tight muon
        eventSelection = tightMuons.counts==1 

        output['muon_pt'].fill(dataset=dataset,
                              pt=tightMuons[eventSelection].pt.flatten())
        
        return output

    def postprocess(self, accumulator):
        return accumulator


In [ ]:
fileset = {"TTGamma":["TTGamma_SingleLept_2016_skimTest.root"],
           "TTbar":["TTbarPowheg_Semilept_2016_skimTest.root"],
          }

output = processor.run_uproot_job(fileset,
                                  treename='Events',
                                  processor_instance=TTGammaCutflow(),
                                  executor=processor.futures_executor,
                                  executor_args={'workers': 4, 'flatten': True},
                                 )

In [ ]:
hist.plot1d(output['muon_pt'],overlay='dataset')